In [33]:
%maven org.apache.lucene:lucene-core:8.0.0
%maven org.apache.lucene:lucene-queryparser:8.0.0

In [34]:
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

In [35]:
import org.apache.lucene.analysis.Analyzer;
import org.apache.lucene.analysis.standard.StandardAnalyzer;
import org.apache.lucene.document.Document;
import org.apache.lucene.document.Field;
import org.apache.lucene.document.TextField;
import org.apache.lucene.index.DirectoryReader;
import org.apache.lucene.index.IndexReader;
import org.apache.lucene.index.IndexWriter;
import org.apache.lucene.index.IndexWriterConfig;
import org.apache.lucene.queryparser.classic.QueryParser;
import org.apache.lucene.search.IndexSearcher;
import org.apache.lucene.search.Query;
import org.apache.lucene.search.ScoreDoc;
import org.apache.lucene.search.TopDocs;
import org.apache.lucene.store.Directory;
import org.apache.lucene.store.FSDirectory;
import org.apache.lucene.util.IOUtils;

In [36]:
Path indexPath = Files.createTempDirectory("tempIndex");
try(Directory dir = FSDirectory.open(indexPath)) {
  Analyzer analyzer = new StandardAnalyzer();
  IndexWriterConfig iwc = new IndexWriterConfig(analyzer);

  try(IndexWriter iwriter = new IndexWriter(dir, iwc)) {
    Document doc = new Document();
    String text = "This is the text to be indexed.";
    doc.add(new Field("fieldname", text, TextField.TYPE_STORED));
    iwriter.addDocument(doc);
  }

  // Now search the index:
  try(DirectoryReader ireader = DirectoryReader.open(dir)) {
    IndexSearcher isearcher = new IndexSearcher(ireader);
    // Parse a simple query that searches for "text":
    QueryParser parser = new QueryParser("fieldname", analyzer);
    Query query = parser.parse("text");
    ScoreDoc[] hits = isearcher.search(query, 10).scoreDocs;
    System.out.println(hits.length);
    // Iterate through the results:
    for (int i = 0; i < hits.length; i++) {
      Document hitDoc = isearcher.doc(hits[i].doc);
      System.out.println(hitDoc.get("fieldname"));
    }
  }
}
IOUtils.rm(indexPath);

1
This is the text to be indexed.
